In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
# TfidfVectorizer - This is used to convert text data into numerical values
from sklearn.metrics.pairwise import cosine_similarity
import os


In [2]:
display (os.getcwd())

'/content'

In [3]:
movies_data = pd.read_csv('/content/movies.csv')
movies_data.head()


,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [4]:
display (movies_data.shape)

(4803, 24)

In [5]:
selected_features = ['original_title','genres','overview','keywords','tagline','cast','director']
print(selected_features)


['original_title', 'genres', 'overview', 'keywords', 'tagline', 'cast', 'director']


In [6]:
display (movies_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

None

In [7]:
display (movies_data.isna().sum())

index                      0
budget                     0
genres                    28
homepage                3091
id                         0
keywords                 412
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                      43
crew                       0
director                  30
dtype: int64

In [8]:
movies = movies_data[selected_features]

In [9]:

movies.head()

,original_title,genres,overview,keywords,tagline,cast,director
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Spectre,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [10]:
movies.isna().sum()

original_title      0
genres             28
overview            3
keywords          412
tagline           844
cast               43
director           30
dtype: int64

In [13]:
for feature in movies:
  movies[feature] = movies[feature].fillna('')
display (movies.head())

<ipython-input-13-5d197b89090a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies[feature] = movies[feature].fillna('')


,original_title,genres,overview,keywords,tagline,cast,director
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski
2,Spectre,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,spy based on novel secret agent sequel mi6,A Plan No One Escapes,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes
3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,dc comics crime fighter terrorist secret ident...,The Legend Ends,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan
4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",based on novel mars medallion space travel pri...,"Lost in our world, found in another.",Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton


In [14]:
movies.shape

(4803, 7)

In [18]:
mov = movies.copy()

In [19]:
mov.head(2)

,original_title,genres,overview,keywords,tagline,cast,director
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski


In [24]:
mov['combined_features'] = mov['genres']+' '+mov['overview']+' '+mov['cast']+' '+mov['director']+' '+mov['tagline']+' '+mov['keywords']
mov['combined_features']

0       Action Adventure Fantasy Science Fiction In th...
1       Adventure Fantasy Action Captain Barbossa, lon...
2       Action Adventure Crime A cryptic message from ...
3       Action Crime Drama Thriller Following the deat...
4       Action Adventure Science Fiction John Carter i...
                              ...                        
4798    Action Crime Thriller El Mariachi just wants t...
4799    Comedy Romance A newlywed couple's honeymoon i...
4800    Comedy Drama Romance TV Movie "Signed, Sealed,...
4801     When ambitious New York attorney Sam is sent ...
4802    Documentary Ever since the second grade when h...
Name: combined_features, Length: 4803, dtype: object

In [26]:
mov['combined_features'][0]

'Action Adventure Fantasy Science Fiction In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Sam Worthington Zoe Saldana Sigourney Weaver Stephen Lang Michelle Rodriguez James Cameron Enter the World of Pandora. culture clash future space war space colony society'

In [27]:
mov.head(2)

,original_title,genres,overview,keywords,tagline,cast,director,combined_features
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",culture clash future space war space colony so...,Enter the World of Pandora.,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,Action Adventure Fantasy Science Fiction In th...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",ocean drug abuse exotic island east india trad...,"At the end of the world, the adventure begins.",Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,"Adventure Fantasy Action Captain Barbossa, lon..."


In [28]:
newmov = mov.drop(columns=['overview','genres','keywords','tagline','director','cast'])

In [29]:
newmov.head(5)

,original_title,combined_features
0,Avatar,Action Adventure Fantasy Science Fiction In th...
1,Pirates of the Caribbean: At World's End,"Adventure Fantasy Action Captain Barbossa, lon..."
2,Spectre,Action Adventure Crime A cryptic message from ...
3,The Dark Knight Rises,Action Crime Drama Thriller Following the deat...
4,John Carter,Action Adventure Science Fiction John Carter i...


In [30]:
nnnn = newmov.copy()

In [31]:
nnnn.head(2)

,original_title,combined_features
0,Avatar,Action Adventure Fantasy Science Fiction In th...
1,Pirates of the Caribbean: At World's End,"Adventure Fantasy Action Captain Barbossa, lon..."


## **Avanced PREPROCESSING**

-------------------------------------------------

# **Using tfidfvectorizer**

In [32]:
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(newmov['combined_features'])
display (feature_vectors.shape)
print (feature_vectors)


(4803, 30592)

  (0, 25232)	0.1336739112380023
  (0, 5592)	0.17322386697661618
  (0, 29469)	0.08637114089261566
  (0, 25413)	0.24369151759694266
  (0, 10796)	0.11095111375730655
  (0, 5279)	0.1549075340655008
  (0, 6601)	0.1498786462809525
  (0, 19410)	0.0318696249813729
  (0, 30121)	0.07272849549697494
  (0, 9065)	0.14505772980982928
  (0, 4288)	0.13213124585063996
  (0, 14180)	0.08418056281586364
  (0, 23134)	0.1498786462809525
  (0, 17784)	0.12285843797047787
  (0, 15569)	0.171366691592621
  (0, 25868)	0.11590870590502833
  (0, 29614)	0.15064979633862852
  (0, 24802)	0.158925784923944
  (0, 23578)	0.16802034155542864
  (0, 30543)	0.15064979633862852
  (0, 30145)	0.18243919685121024
  (0, 23619)	0.11047375838269538
  (0, 5245)	0.16506865163441015
  (0, 1014)	0.12285843797047787
  (0, 1243)	0.05270709034262741
  :	:
  (4802, 27153)	0.05940079357098276
  (4802, 19243)	0.061349419053434766
  (4802, 11941)	0.07829183421073846
  (4802, 29680)	0.07735547381237222
  (4802, 9346)	0.07104321122746965
  (480

In [33]:
similarity = cosine_similarity(feature_vectors)
print (similarity )


[[1.         0.05083168 0.0332947  ... 0.02749812 0.0304889  0.0072518 ]
 [0.05083168 1.         0.04356836 ... 0.05077045 0.03100979 0.01521198]
 [0.0332947  0.04356836 1.         ... 0.02646984 0.04751623 0.01372603]
 ...
 [0.02749812 0.05077045 0.02646984 ... 1.         0.03481447 0.03546821]
 [0.0304889  0.03100979 0.04751623 ... 0.03481447 1.         0.03098945]
 [0.0072518  0.01521198 0.01372603 ... 0.03546821 0.03098945 1.        ]]


In [34]:
display(similarity.shape)

(4803, 4803)

In [36]:
m = pd.DataFrame (similarity )
m.head()

,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.050832,0.033295,0.034801,0.087247,0.056697,0.022512,0.067737,0.021355,0.033355,...,0.008694,0.008800,0.038022,0.043411,0.018550,0.031712,0.008506,0.027498,0.030489,0.007252
1,0.050832,1.000000,0.043568,0.035838,0.081385,0.054802,0.034643,0.089591,0.064149,0.050724,...,0.015338,0.022036,0.037701,0.041613,0.047268,0.040357,0.016923,0.050770,0.031010,0.015212
2,0.033295,0.043568,1.000000,0.039562,0.035718,0.033357,0.016806,0.076981,0.035585,0.032055,...,0.039963,0.002932,0.015055,0.011467,0.030193,0.026837,0.005150,0.026470,0.047516,0.013726
3,0.034801,0.035838,0.039562,1.000000,0.030266,0.031565,0.037230,0.064623,0.042408,0.123724,...,0.019504,0.013498,0.044522,0.034591,0.027855,0.037315,0.014054,0.048695,0.047812,0.025365
4,0.087247,0.081385,0.035718,0.030266,1.000000,0.070102,0.042003,0.085112,0.022993,0.055120,...,0.019824,0.025665,0.033504,0.041296,0.062263,0.043023,0.009632,0.045532,0.030357,0.019229


In [37]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : avenger


In [41]:
list_of_all_titles = newmov['original_title'].tolist()
print(list_of_all_titles)


['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [43]:
len(list_of_all_titles)

4803

In [44]:

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)


['The Avengers']


In [45]:
close_match = find_close_match[0]
print(close_match)

The Avengers


In [46]:
index_of_the_movie = newmov[newmov.original_title == close_match].index.values[0]
print(index_of_the_movie)


16


In [47]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)


[(0, 0.04270511084643852), (1, 0.06224849736436751), (2, 0.038497022076105974), (3, 0.03866029496208938), (4, 0.07911855483427595), (5, 0.028219968789126127), (6, 0.03223768203133271), (7, 0.4402353799961129), (8, 0.05694612067285468), (9, 0.0900244307215416), (10, 0.07257687378091392), (11, 0.02424446248510516), (12, 0.021581747144618833), (13, 0.030140555006309223), (14, 0.12280045566128427), (15, 0.044125602745461394), (16, 1.0), (17, 0.062176884685858205), (18, 0.09406180183659207), (19, 0.046170896553890076), (20, 0.07491686150195935), (21, 0.05098295245244174), (22, 0.029998128563312777), (23, 0.040393533749242094), (24, 0.039607990050449945), (25, 0.029469554253520047), (26, 0.20395712302219837), (27, 0.07126731298189382), (28, 0.056682918402660334), (29, 0.06763120531181947), (30, 0.0864695759564071), (31, 0.13988291470369982), (32, 0.047082947149254435), (33, 0.13338190411264159), (34, 0.014097601041070489), (35, 0.03929614190478607), (36, 0.056390138760334226), (37, 0.0306269

In [48]:

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)


[(16, 1.0), (7, 0.4402353799961129), (85, 0.2203039100288559), (79, 0.21875024386891997), (26, 0.20395712302219837), (511, 0.1822759407731296), (174, 0.17132075087923182), (126, 0.165340198355461), (64, 0.16420929069328924), (169, 0.1463207117759176), (46, 0.14225094695565857), (182, 0.1401795608196723), (31, 0.13988291470369982), (33, 0.13338190411264159), (68, 0.13208839109747222), (101, 0.12943581971790338), (129, 0.12929891610324504), (203, 0.12580808444072306), (14, 0.12280045566128427), (38, 0.12056450312255525), (1720, 0.11700903976544032), (870, 0.11672626627773836), (1294, 0.11388770846814351), (94, 0.11336879003815323), (1230, 0.1125643783949835), (215, 0.11113767691337564), (788, 0.10986510020201289), (1892, 0.1091073911294834), (782, 0.10737096085522715), (713, 0.10679787508044816), (4401, 0.10525669069412166), (1569, 0.10467168495538937), (242, 0.1031335794946852), (4759, 0.10017500295475637), (2874, 0.09897325494600731), (1286, 0.09838660260231001), (256, 0.09764183467804

In [49]:

print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  # print(index)
  title_from_index = newmov[newmov.index==index]['original_title'].values
  if (i<30):
    print(i, '.',title_from_index)
    i+=1


Movies suggested for you : 

1 . ['The Avengers']
2 . ['Avengers: Age of Ultron']
3 . ['Captain America: The Winter Soldier']
4 . ['Iron Man 2']
5 . ['Captain America: Civil War']
6 . ['X-Men']
7 . ['The Incredible Hulk']
8 . ['Thor: The Dark World']
9 . ['X-Men: Apocalypse']
10 . ['Captain America: The First Avenger']
11 . ['X-Men: Days of Future Past']
12 . ['Ant-Man']
13 . ['Iron Man 3']
14 . ['X-Men: The Last Stand']
15 . ['Iron Man']
16 . ['X-Men: First Class']
17 . ['Thor']
18 . ['X2']
19 . ['Man of Steel']
20 . ['The Amazing Spider-Man 2']
21 . ['Kick-Ass']
22 . ['Superman II']
23 . ['Serenity']
24 . ['Guardians of the Galaxy']
25 . ['The Shadow']
26 . ['4: Rise of the Silver Surfer']
27 . ['Deadpool']
28 . ['The Losers']
29 . ['The Spirit']


In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

In [51]:
documents = ["doc1.txt", "doc2.txt", "doc3.txt"]


## raw documents to tf-idf matrix:

In [52]:
vectorizer = TfidfVectorizer(stop_words='english', use_idf=True, smooth_idf=True)

SVD to reduce dimensionality:

In [53]:
svd_model = TruncatedSVD(n_components=100, algorithm='randomized', n_iter=10)

pipeline of tf-idf + SVD, fit to and applied to documents:

In [54]:
svd_transformer = Pipeline([('tfidf', vectorizer), ('svd', svd_model)])

In [55]:
svd_matrix = svd_transformer.fit_transform(newmov['combined_features'])


In [56]:
display (svd_matrix.shape)
print (svd_matrix)

(4803, 100)

[[ 0.12114434  0.18342507  0.09879424 ...  0.01453564 -0.05419544
  -0.01492102]
 [ 0.1366692   0.04568989  0.0609464  ...  0.00104057  0.03697748
  -0.02343157]
 [ 0.10474134  0.10187264 -0.0381828  ...  0.02025615 -0.00041188
  -0.04022847]
 ...
 [ 0.11617644 -0.0212411   0.03296889 ...  0.00518638 -0.00256423
   0.00783385]
 [ 0.0841176  -0.02265375 -0.01293717 ... -0.00723475 -0.03460428
  -0.03309573]
 [ 0.0670773  -0.0305071  -0.01280458 ...  0.01037617  0.04985578
   0.06128371]]


In [57]:
similarity = cosine_similarity(svd_matrix)
print  (similarity )

[[ 1.00000000e+00  1.52299362e-01  1.76167376e-01 ...  9.99094960e-02
   1.21450144e-01  5.24654497e-02]
 [ 1.52299362e-01  1.00000000e+00  3.70359765e-02 ...  3.67955243e-02
   9.39360857e-02  1.45532003e-03]
 [ 1.76167376e-01  3.70359765e-02  1.00000000e+00 ...  8.31805398e-02
   2.09248472e-01 -1.19014143e-04]
 ...
 [ 9.99094960e-02  3.67955243e-02  8.31805398e-02 ...  1.00000000e+00
   1.99996871e-01  2.50158513e-01]
 [ 1.21450144e-01  9.39360857e-02  2.09248472e-01 ...  1.99996871e-01
   1.00000000e+00  8.49146683e-02]
 [ 5.24654497e-02  1.45532003e-03 -1.19014143e-04 ...  2.50158513e-01
   8.49146683e-02  1.00000000e+00]]


In [58]:
display(similarity.shape)

(4803, 4803)

In [59]:
m = pd.DataFrame (similarity )
m.head(2)

,0,1,2,3,4,5,6,7,8,9,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
0,1.000000,0.152299,0.176167,0.005188,0.630375,0.215331,0.144629,0.155043,0.004995,0.125985,...,0.112751,0.057263,0.035155,0.176895,0.006761,0.074469,0.032298,0.099909,0.121450,0.052465
1,0.152299,1.000000,0.037036,0.037214,0.365636,0.219319,0.178320,0.227217,0.106145,0.142547,...,0.164715,0.049098,0.141173,0.083483,0.026160,0.125634,0.045803,0.036796,0.093936,0.001455


In [67]:
list_of_all_titles = newmov['original_title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [68]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : batman


In [69]:

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)


['Batman', 'Batman', 'Catwoman']


In [70]:
close_match = find_close_match[0]
print(close_match)

Batman


In [71]:
index_of_the_movie = newmov[newmov.original_title == close_match].index.values[0]
print(index_of_the_movie)

1359


In [72]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)


[(0, 0.05395648496339094), (1, 0.19339361968080723), (2, 0.0971177441350791), (3, 0.715156395500932), (4, 0.03104465635494705), (5, 0.15650870766343802), (6, 0.0561335299388207), (7, 0.07638151582590352), (8, 0.24732119925194476), (9, 0.5078810382773695), (10, 0.3712781259735928), (11, 0.14561443312082642), (12, 0.2621928771541396), (13, 0.08279636772803387), (14, 0.3262847292379946), (15, 0.0955744848100388), (16, 0.07317680302013106), (17, 0.19344039746214292), (18, 0.11870770947745171), (19, 0.10859280150994069), (20, 0.13682850285985823), (21, 0.13603150681139245), (22, 0.09599301207019581), (23, 0.11381669019208113), (24, 0.1092277812919947), (25, 0.11100310173387187), (26, 0.12352555022349206), (27, 0.048376118379901654), (28, -0.00793211426189601), (29, 0.03178408810509508), (30, 0.31803871005448503), (31, 0.1408455865471492), (32, 0.12572175701724617), (33, 0.06208588876690207), (34, -0.026720226051844438), (35, 0.3378444325127506), (36, 0.3919096131448495), (37, 0.127850579515

In [73]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(1359, 0.9999999999999999), (428, 0.8254836219159469), (65, 0.7621666831879749), (119, 0.7434705753897246), (210, 0.7328806810961322), (3, 0.715156395500932), (299, 0.592202706948606), (4065, 0.5180245752321052), (3854, 0.5107013572438929), (9, 0.5078810382773695), (3819, 0.4974426692588049), (72, 0.479493301259957), (4778, 0.46848406574503326), (4638, 0.4676507694847502), (813, 0.45905921039791786), (2001, 0.4587833911552858), (3409, 0.447986880997975), (2313, 0.44769959080320354), (3359, 0.4348487868185432), (1001, 0.4186535025719276), (4313, 0.41789475534326886), (3306, 0.41460876862624924), (303, 0.412048163619081), (1192, 0.4059752693640344), (3020, 0.40565189395542184), (826, 0.40472822221827526), (3515, 0.40019813136945254), (2492, 0.39499052095794895), (36, 0.3919096131448495), (2130, 0.39012541796330646), (2530, 0.384417052072986), (3454, 0.38435468417029806), (4440, 0.3830522072554482), (782, 0.3811397287995859), (4142, 0.38050850383825296), (3337, 0.3792167077106825), (1024

In [74]:
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1


Movies suggested for you : 

1 . Batman
2 . Batman Returns
3 . The Dark Knight
4 . Batman Begins
5 . Batman & Robin
6 . The Dark Knight Rises
7 . Batman Forever
8 . Mi America
9 . Batman: The Dark Knight Returns, Part 2
10 . Batman v Superman: Dawn of Justice
11 . Defendor
12 . Suicide Squad
13 . Rampage
14 . Amidst the Devil's Wings
15 . Superman
16 . The Crew
17 . Criminal Activities
18 . The Mask
19 . In Too Deep
20 . Street Fighter: The Legend of Chun-Li
21 . Below Zero
22 . Code of Honor
23 . Catwoman
24 . Spawn
25 . Sugar Hill
26 . Disclosure
27 . Freaky Deaky
28 . Steel
29 . Transformers: Age of Extinction


In [75]:
          # movie_name = input(' Enter your favourite movie name : ')

          # list_of_all_titles = movies_data['title'].tolist()

          # find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

          # close_match = find_close_match[0]

          # index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

          # similarity_score = list(enumerate(similarity[index_of_the_movie]))

          # sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

          # print('Movies suggested for you : \n')

          # i = 1

          # for movie in sorted_similar_movies:
          #   index = movie[0]
          #   title_from_index = movies_data[movies_data.index==index]['title'].values[0]
          #   if (i<30):
          #     print(i, '.',title_from_index)
          #     i+=1

### **using bag of words**

In [76]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [77]:
vector = cv.fit_transform(nnnn['combined_features']).toarray()

In [78]:
vector.shape

(4803, 5000)

In [79]:
from sklearn.metrics.pairwise import cosine_similarity


In [80]:
similarity = cosine_similarity(vector)

In [81]:
similarity

array([[1.        , 0.1153164 , 0.07150969, ..., 0.02132007, 0.04662524,
        0.        ],
       [0.1153164 , 1.        , 0.06596992, ..., 0.05900529, 0.02150662,
        0.        ],
       [0.07150969, 0.06596992, 1.        , ..., 0.02032789, 0.08891084,
        0.        ],
       ...,
       [0.02132007, 0.05900529, 0.02032789, ..., 1.        , 0.05964321,
        0.04767313],
       [0.04662524, 0.02150662, 0.08891084, ..., 0.05964321, 1.        ,
        0.0521286 ],
       [0.        , 0.        , 0.        , ..., 0.04767313, 0.0521286 ,
        1.        ]])

In [82]:
similarity.shape

(4803, 4803)

In [83]:
movie_name = input(' Enter your favourite movie name : ')

 Enter your favourite movie name : avenger


In [84]:
list_of_all_titles = nnnn['original_title'].tolist()

In [85]:
find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

In [86]:
close_match = find_close_match[0]

In [87]:
close_match

'The Avengers'

In [88]:
index_of_the_movie = newmov[newmov.original_title == close_match].index.values[0]
print(index_of_the_movie)

16


In [89]:
similarity_score = list(enumerate(similarity[index_of_the_movie]))

In [90]:
sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

In [91]:
print('Movies suggested for you : \n')

i = 1

for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = nnnn[nnnn.index==index]['original_title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Iron Man 2
4 . Captain America: Civil War
5 . X-Men
6 . The Incredible Hulk
7 . Captain America: The Winter Soldier
8 . Ant-Man
9 . Thor: The Dark World
10 . X-Men: Apocalypse
11 . Captain America: The First Avenger
12 . Iron Man 3
13 . X2
14 . X-Men: The Last Stand
15 . Iron Man
16 . X-Men: First Class
17 . Allegiant
18 . The Day After Tomorrow
19 . Guardians of the Galaxy
20 . Fantastic Four
21 . Man of Steel
22 . The Shadow
23 . The Helix... Loaded
24 . Down to Earth
25 . Deadpool
26 . X-Men: Days of Future Past
27 . 4: Rise of the Silver Surfer
28 . Superman II
29 . London
